In [10]:
import numpy as np
import torch
import torch.nn as nn
from bayesian_torch.layers import Conv2dFlipout, LinearFlipout
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from tqdm import tqdm
from sklearn.model_selection import train_test_split


In [ ]:
# 主要参数设置
epochs = 5
complex_cost = 1e-5
batch_size = 40

读取样本数据

In [ ]:
signals = np.load('signals.npy')
labels = np.load('time_labels.npy')
signals = signals.transpose(0,2,1)

signals = signals[:,np.newaxis,:,:]
labels = labels[:, np.newaxis]
print(signals.shape)
print(labels.shape)

test_size = 0.2  # 测试集比例
random_state = 42

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(signals, labels, test_size=test_size, random_state=random_state)

print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False)

(344, 1, 32768, 2)
(344, 1)
Training set size: 275
Testing set size: 69


构建模型

In [12]:
class BayesianLargeKernelAttention(nn.Module):
    def __init__(self, in_channels, kernel_size, dilation_rate=1, reduction_ratio=16):
        super(BayesianLargeKernelAttention, self).__init__()
        
        self.kl_divergence = 0.0
        # 贝叶斯深度卷积
        self.depthwise_conv = Conv2dFlipout(
            in_channels=in_channels,
            out_channels=in_channels,
            kernel_size=(kernel_size, 1),
            stride=(1, 1),
            padding=2,
            groups=in_channels,
            prior_variance=0.1
        )
        
        # 贝叶斯深度膨胀卷积
        self.dilated_conv = Conv2dFlipout(
            in_channels=in_channels,
            out_channels=in_channels,
            kernel_size=(7, 1),
            stride=(1, 1),
            padding=9,
            groups=in_channels,
            prior_variance=0.1,
            bias=True
        )
        
        # 贝叶斯逐点卷积
        self.pointwise_conv = Conv2dFlipout(
            in_channels=in_channels,
            out_channels=in_channels // reduction_ratio,
            kernel_size=1,
            stride=(1, 1),
            prior_variance=0.1,
            bias=True
        )
        
        # 1×1卷积核获得注意力权重
        self.attention_conv = Conv2dFlipout(
            in_channels=in_channels // reduction_ratio,
            out_channels=in_channels,
            kernel_size=1
        )
        
        self.bn = nn.BatchNorm2d(in_channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        kl_total = 0.0
        x_depthwise, cur_kl = self.depthwise_conv(x)
        kl_total += cur_kl
        x_dilated, cur_kl = self.dilated_conv(x_depthwise)
        kl_total += cur_kl
        x_pointwise, cur_kl = self.pointwise_conv(x_dilated)
        kl_total += cur_kl
        attention_weights, cur_kl = self.attention_conv(x_pointwise)  
        kl_total += cur_kl     
        attention_weights = self.sigmoid(self.bn(attention_weights))
        batch, channel, height, width = x.shape
        attention_weights = nn.functional.interpolate(attention_weights, size=(height, width), mode='bilinear')
        attended_features = x * attention_weights
        self.kl_divergence += kl_total
        
        return attended_features

class regressor(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(regressor, self).__init__()
        self.flatten = nn.Flatten()
        self.linear1 = LinearFlipout(in_channels, 100)
        self.linear2 = LinearFlipout(100, out_channels)
        self.relu = nn.ReLU()
        self.kl_divergence = 0.0
        
    def forward(self, x):
        kl_total = 0.0
        x = self.flatten(x)
        x, cur_kl = self.linear1(x)
        kl_total += cur_kl
        x, cur_kl = self.linear2(x)
        kl_total += cur_kl
        x = self.relu(x)
        self.kl_divergence += kl_total
        return x

class BLKAN(nn.Module):
    def __init__(self, in_channels=1, num_classes=1):
        super(BLKAN, self).__init__()
        
        # 特征提取
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels, 24, kernel_size=(5, 1), stride=(1, 1), padding=(2, 0)),
            nn.BatchNorm2d(24),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=(5, 1), stride=(5, 1)),
            BayesianLargeKernelAttention(24, kernel_size=5, dilation_rate=3),
            nn.AvgPool2d(kernel_size=(5, 1), stride=(5, 1)),

            BayesianLargeKernelAttention(24, kernel_size=5, dilation_rate=3),
            nn.AvgPool2d(kernel_size=(5, 1), stride=(5, 1)),

            BayesianLargeKernelAttention(24, kernel_size=5, dilation_rate=3),
            nn.AvgPool2d(kernel_size=(5, 1), stride=(5, 1)),

            BayesianLargeKernelAttention(24, kernel_size=7, dilation_rate=3),
            nn.AvgPool2d(kernel_size=(5, 1), stride=(5, 1)),

            BayesianLargeKernelAttention(24, kernel_size=5, dilation_rate=3)
        )

        # 回归分析
        self.regressor = regressor(480, num_classes)

    def forward(self, x):
        features = self.feature_extractor(x)
        flattened_features = torch.cat([features.view(features.size(0), -1)], dim=1)
        output = self.regressor(flattened_features)
        return output



定义损失函数

In [ ]:

def elbo_loss(model, x, y, num_samples=3):
    mse_loss_fn = nn.MSELoss(reduction='sum')
    kl_divergence = 0

    total_mse = 0
    for _ in range(num_samples):
        output = model(x)
        total_mse += mse_loss_fn(output, y)
    
    # 计算 KL 散度
    for module in model.modules():
        if hasattr(module, 'kl_divergence'):
            kl_divergence += module.kl_divergence
            module.kl_divergence = 0.0
            print(module.kl_divergence)

    nll = total_mse / num_samples
    elbo = nll + kl_divergence / num_samples / batch * complex_cost  # 调整复杂性成本权重
    return elbo

# 初始化模型和优化器
model = BLKAN(num_classes=1)
optimizer = optim.Adam(model.parameters(), lr=0.005)

训练模型

In [15]:
def train_model(model, train_loader, optimizer, num_epochs=100):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch_x, batch_y in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            optimizer.zero_grad()
            loss = elbo_loss(model, batch_x, batch_y)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

train_model(model, train_loader, optimizer, epochs)

Epoch 1/5:   0%|          | 0/7 [00:00<?, ?it/s]

0.0
0.0
0.0
0.0
0.0
0.0


Epoch 1/5:  14%|█▍        | 1/7 [00:58<05:49, 58.22s/it]

0.0
0.0
0.0
0.0
0.0
0.0


Epoch 1/5:  29%|██▊       | 2/7 [01:58<04:53, 58.77s/it]

...

0.0
0.0
0.0
0.0
0.0
0.0


Epoch 5/5:  86%|████████▌ | 6/7 [05:58<00:59, 59.90s/it]

0.0
0.0
0.0
0.0
0.0
0.0


Epoch 5/5: 100%|██████████| 7/7 [06:48<00:00, 58.42s/it]

Epoch [5/5], Loss: 3.0806


测试并计算不确定度

In [17]:
def predict_with_uncertainty(model, test_loader, num_samples=50):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch_x, _ in test_loader:
            batch_predictions = []
            for _ in range(num_samples):
                output = model(batch_x)
                batch_predictions.append(output.numpy())
            batch_predictions = np.stack(batch_predictions, axis=0)
            mean_prediction = batch_predictions.mean(axis=0)
            var_prediction = batch_predictions.var(axis=0)
            predictions.append((mean_prediction, var_prediction))
    return predictions

predictions = predict_with_uncertainty(model, test_loader)

# 解析预测结果
for i, (mean_pred, var_pred) in enumerate(predictions):
    print(f'Sample {i}: Mean Prediction = {mean_pred}, Variance = {var_pred}')

Sample 0: Mean Prediction = [[0.42428714]
 [0.4157982 ]
 [0.40506482]
 [0.4219145 ]
 [0.41825202]
 [0.43552005]
 [0.41913083]
 [0.4236773 ]
 [0.41429615]
 [0.44044605]
 [0.42041877]
 [0.39153376]
 [0.36743584]
 [0.41366485]
 [0.42786035]
 [0.3961564 ]
 [0.3863121 ]
 [0.42643124]
 [0.43319622]
 [0.39831933]
 [0.41711932]
 [0.41009358]
 [0.42167482]
 [0.39797807]
 [0.42978978]
 [0.41676602]
 [0.41054633]
 [0.45424262]
 [0.4297251 ]
 [0.40934327]
 [0.41075104]
 [0.40923366]
 [0.42754886]
 [0.42552796]
 [0.40106693]
 [0.4034327 ]
 [0.43090492]
 [0.4305793 ]
 [0.41415742]
 [0.40676853]], Variance = [[0.00515517]
 [0.00632663]
 [0.00429259]
 [0.006656  ]
 [0.00428488]
 [0.00633378]
 [0.00419422]
 [0.00565914]
 [0.00750319]
 [0.00650128]
 [0.0062028 ]
 [0.00557311]
 [0.0035007 ]
 [0.00590376]
 [0.00648958]
 [0.00469124]
 [0.00594695]
 [0.00730249]
 [0.00684249]
 [0.00623543]
 [0.00511617]
 [0.0063838 ]
 [0.00622635]
 [0.00709957]
 [0.00426448]
 [0.00596195]
 [0.00505201]
 [0.00505636]
 [0.006